<a href="https://colab.research.google.com/github/lbhagavan/stanford_LLM_Leela/blob/homework/%5Bdanrl%5D_TECH16_Homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all the things
%pip install llama_index
%pip install textwrap3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# Housekeeping
import textwrap3
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["USER_AGENT"] = 'TECH16 Colab'

# Helper
def wrap_print(long_text):
  print('\n'.join(textwrap3.wrap(long_text)))

# Load Documents

In [ ]:
from llama_index.core import Document
import glob

# Download the state and county building codes.
# What an exciting Sunday afternoon read those are!
!rm -rf /tmp/codes
!mkdir -p /tmp/codes
!wget https://x.gl/SantaCruzCountyCode.txt -O /tmp/codes/scc.txt
!wget https://x.gl/cacode.tar.gz -O - | tar -xzv -C /tmp/codes/

docs = []
for fname in glob.glob("/tmp/codes/*.txt"):
  with open(fname, 'r') as file:
    docs.append(Document(text=file.read()))

# OMG, what have we done?
#print(doc)

--2024-08-02 02:48:44--  https://x.gl/SantaCruzCountyCode.txt
Resolving x.gl (x.gl)... 104.21.32.111, 172.67.151.19, 2606:4700:3036::ac43:9713, ...
Connecting to x.gl (x.gl)|104.21.32.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘/tmp/codes/scc.txt’

/tmp/codes/scc.txt      [  <=>               ]   8.01M  32.9MB/s    in 0.2s    

2024-08-02 02:48:44 (32.9 MB/s) - ‘/tmp/codes/scc.txt’ saved [8402226]

--2024-08-02 02:48:45--  https://x.gl/cacode.tar.gz
Resolving x.gl (x.gl)... 104.21.32.111, 172.67.151.19, 2606:4700:3036::ac43:9713, ...
Connecting to x.gl (x.gl)|104.21.32.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7540377 (7.2M) [application/gzip]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               ._2022californiaad00unse_djvu.txt
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
2022californiaad00uns

# Build Index

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage

# nom nom nom, consume all the tokens
#index = VectorStoreIndex.from_documents(docs)

# or maybe save some money and re-use the index from last time :)
!rm -rf /tmp/index/ && mkdir -p /tmp/index/
!wget https://x.gl/SantaCruzCountyCode.index.tar.gz.part_aa -O a && \
 wget https://x.gl/SantaCruzCountyCode.index.tar.gz.part_ab -O b
!cat a b | tar -xzv -C /tmp/index/
storage_context = StorageContext.from_defaults(persist_dir='/tmp/index/')
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()

# Now we can talk to the building code :)

In [ ]:
wrap_print(query_engine.query(
    "What are the requirements for a septic tank for a three bedroom house?",
).response)

The minimum septic tank volumetric capacity serving residences with
one to four bedrooms shall be 1,500 gallons. Additionally, septic
tanks for residences with more than four bedrooms should have an
additional 250 gallons of capacity per bedroom. The septic tank should
have at least two compartments separated by a baffle or equivalent
arrangement, with the inlet compartment having a capacity of not less
than two-thirds of the total volume. Access to each compartment should
be provided by a manhole 20 inches in minimum dimensions with a close-
fitting manhole cover equipped with a durable handle for removal.


In [ ]:
wrap_print(query_engine.query(
  "Can I choose my own house number for my new home?",
).response)

You cannot choose your own house number for your new home. The
assignment of street names and building numbers is typically done by
the Director of Emergency Services or a designated authority based on
certain criteria outlined in the regulations.


In [ ]:
wrap_print(query_engine.query(
  "Any fire prevention measures I need to keep in mind when building a house?",
).response)

Ensure that any installed gas fireplace is a direct-vent sealed-
combustion type. Any installed woodstove or pellet stove should comply
with US EPA New Source Performance Standards emission limits and have
a permanent label indicating certification. Additionally, woodstoves,
pellet stoves, and fireplaces must comply with applicable local
ordinances. Cover duct openings and protect mechanical equipment
during construction to prevent the entry of water, dust, and debris
into the system. Finish materials used should meet the specified
requirements to control pollutants.


In [ ]:
wrap_print(query_engine.query(
  "What are the requirements for a solar panel shutdown system in residential homes? What about rapid shutdown?",
).response)

Solar panel shutdown systems in residential homes must include a rapid
shutdown switch with a label that specifies "RAPID SHUTDOWN SWITCH FOR
SOLAR PV SYSTEM." The label should be reflective, with all letters
capitalized and a minimum height of 9.5 mm in white on a red
background. Additionally, for buildings with multiple rapid shutdown
types or no rapid shutdown, a detailed plan view diagram of the roof
must be provided showing each different PV system with a dotted line
around areas that remain energized after rapid shutdown is initiated.
